In [1]:
# Importações

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import folium


import json



from src.config import CLEANED_PATH, GEO_CALIFORNIA_ORIGINAL_DIR, AGG_COUNTIES_DIR

In [2]:
# Definição do tema do seaborn
sns.set_theme(
    style="white",
    palette="bright"
)

In [ ]:
# Obtendo a base de dados
df = pd.read_parquet(CLEANED_PATH)

df.head()

In [ ]:
# Resumo da base
df.info()

In [ ]:
# Resumo estatístico da base
df.describe()

In [ ]:
# Verificando o comportamento e a concentração do par longitude e latitude

sns.jointplot(
    data=df, 
    x="longitude", 
    y="latitude",
    alpha=0.5
)

plt.show()

In [ ]:
# Verificando o comportamento do par longitude e latitude, para cada categoria de "ocean_proximity"

fig, ax = plt.subplots()

sns.scatterplot(
    data=df, 
    x="longitude", 
    y="latitude",
    hue="ocean_proximity",
    alpha=0.15,
    ax=ax
)

plt.show()

In [ ]:
# Verificando o comportamento do par longitude e latitude, para cada categoria de "median_income_cat"

fig, ax = plt.subplots()

sns.scatterplot(
    data=df, 
    x="longitude", 
    y="latitude",
    hue="median_income_cat",
    ax=ax,
    palette="coolwarm"
)

plt.show()

In [ ]:
# Verificando o comportamento do par longitude e latitude, para cada categoria de "housing_median_age_cat"

fig, ax = plt.subplots()

sns.scatterplot(
    data=df, 
    x="longitude", 
    y="latitude",
    hue="housing_median_age_cat",
    ax=ax,
    palette="coolwarm"
)

plt.show()

In [ ]:
# Verificando o comportamento do par longitude e latitude, para a coluna alvo

norm_median_house_value = plt.Normalize(
    vmin=df["median_house_value"].min(), 
    vmax=df["median_house_value"].max()
    )
sm_median_house_value = plt.cm.ScalarMappable(
    norm=norm_median_house_value,
    cmap="coolwarm"
    )

fig, ax = plt.subplots()

sns.scatterplot(
    data=df, 
    x="longitude", 
    y="latitude",
    hue="median_house_value",
    ax=ax,
    palette="coolwarm"
)

ax.get_legend().remove()

fig.colorbar(mappable=sm_median_house_value, ax=ax)

plt.show()

In [ ]:
# Obtendo os dados dos condados
gdf_counties = gpd.read_file(GEO_CALIFORNIA_ORIGINAL_DIR)
gdf_counties.head()

In [ ]:
# Resumo da base
gdf_counties.info()

In [13]:
# Removendo as colunas desnecessárias
gdf_counties = gdf_counties.drop(
    [
        "cartodb_id", 
        "created_at", 
        "updated_at"
        ], axis=1)

In [ ]:
# Gerando dados geométricos da base de grupos censitários
points = [
    Point(lon, lat) for lon, lat in zip(
        df["longitude"].values, 
        df["latitude"].values
        )
        ]
points[:20]

In [ ]:
# Obtendo os dados dos grupos censitários por geodataframe
gdf = gpd.GeoDataFrame(df, geometry=points)
gdf.head()

In [ ]:
# Verificando o sistema de referencia de coordenada das bases
print(gdf.crs)
print(gdf_counties.crs)

In [ ]:
# Convertendo a base de grupos censitários para o CRS da base dos condados
gdf = gdf.set_crs(epsg=4326)
gdf.crs

In [ ]:
# Juntando os objetos que pertencem a um dado condado
gdf_joined = gpd.sjoin(
    gdf, gdf_counties, 
    how="left", 
    predicate="within"
    )
gdf_joined.head()

In [19]:
# Removendo a coluna "index_right" da base juntada
gdf_joined = gdf_joined.drop("index_right", axis=1)

In [ ]:
# Resumo da base
gdf_joined.info()

In [ ]:
# Verificando os registros com valores nulos
gdf_joined[gdf_joined.isnull().any(axis=1)]

Percebe-se que existem alguns objetos sem condado relacionado. Assim, usaremos uma estratégia de imputação de valores nulos correspondente ao condado mais próximo aquela região. Isso será feito com base no centróide

In [ ]:
# Gerando uma coluna correspondente ao centróide
gdf_counties["centroid"] = gdf_counties["geometry"].centroid

In [ ]:
def getApproachCounty(data: pd.Series, column: str):

    if pd.isna(data[column]):

        distances = gdf_counties["centroid"].distance(data["geometry"])
        less_distance_idx = distances.idxmin()

        return gdf_counties.loc[less_distance_idx][column]

    return data[column]


for column, _ in filter(lambda x: x != 0, gdf_joined.isnull().sum().items()):

    gdf_joined[column] = gdf_joined.apply(lambda x: getApproachCounty(x, column), axis=1)

In [ ]:
# Verificando os registros nulos novamente
gdf_joined[gdf_joined.isnull().any(axis=1)]

In [ ]:
# Acessando um grupo censitário especifico que não tinha condado anteriormente
gdf_joined.loc[79]

In [ ]:
# Gerando mapa da califórnia

fig, ax = plt.subplots(figsize=(10, 10))

gdf_counties.plot(
    edgecolor="black",
    color="gray",
    ax=ax
)

ax.scatter(
    x=gdf_joined["longitude"],
    y=gdf_joined["latitude"],
    color="red",
    s=1,
    alpha=0.5
)

for x, y, name in zip(
    gdf_counties["centroid"].x,
    gdf_counties["centroid"].y,
    gdf_counties["name"]
):
    
    ax.text(x, y, name, fontsize=6, ha="center", va="center")

In [ ]:
# Agrupando os dados numéricos de cada condado
gdf_joined.groupby("name").median(numeric_only=True)

In [ ]:
# Juntando os dados numéricos agregados de cada condado com a base de condados

gdf_counties = gdf_counties.merge(
    gdf_joined.groupby("name").median(numeric_only=True),
    left_on="name",
    right_index=True
)
gdf_counties.head()

In [ ]:
# Agrupando os dados categóricos de cada condado
gdf_joined[["name", "ocean_proximity"]].groupby("name").describe()

In [ ]:
# Agrupando a moda de ocean_proximity para cada condado
counties_ocean_prox = gdf_joined[["name", "ocean_proximity"]].groupby("name").agg(lambda x: x.mode())
counties_ocean_prox.head()

In [31]:
# Juntando os dados categóricos agregados de cada condado na base de condados
gdf_counties = gdf_counties.merge(
    counties_ocean_prox,
    left_on="name",
    right_index=True
)

In [ ]:
gdf_counties.head()

In [ ]:
# Gerando novo mapa com os dados agregados para cada condado da coluna alvo

fig, ax = plt.subplots(figsize=(12, 12))

gdf_counties.plot(
    edgecolor="black",
    column="median_house_value",
    cmap="coolwarm",
    ax=ax,
)

norm_agg_median_house_value = plt.Normalize(
    vmin=gdf_counties["median_house_value"].min(),
    vmax=gdf_counties["median_house_value"].max()
)

sm_agg_median_house_value = plt.cm.ScalarMappable(
    norm=norm_agg_median_house_value,
    cmap="coolwarm",
)

fig.colorbar(
    sm_agg_median_house_value, 
    ax=ax, 
    label="Median House Value"
    )

for x, y, name in zip(
    gdf_counties["centroid"].x, 
    gdf_counties["centroid"].y, 
    gdf_counties["name"]
    ):

    ax.text(x, y, name, fontsize=6, ha="center", va="center")

In [ ]:
# Gerando novo mapa com os dados agregados para cada condado da coluna "median_income"

fig, ax = plt.subplots(figsize=(12, 12))

gdf_counties.plot(
    edgecolor="black",
    column="median_income",
    cmap="coolwarm",
    ax=ax,
)

norm_agg_median_income = plt.Normalize(
    vmin=gdf_counties["median_income"].min(),
    vmax=gdf_counties["median_income"].max()
)

sm_agg_median_income = plt.cm.ScalarMappable(
    norm=norm_agg_median_income,
    cmap="coolwarm",
)

fig.colorbar(
    sm_agg_median_income, 
    ax=ax, 
    label="Median Income"
    )

for x, y, name in zip(
    gdf_counties["centroid"].x, 
    gdf_counties["centroid"].y, 
    gdf_counties["name"]
    ):

    ax.text(x, y, name, fontsize=6, ha="center", va="center")

In [ ]:
# Gerando novo mapa com os dados agregados para cada condado da coluna "housing_median_age"

fig, ax = plt.subplots(figsize=(12, 12))

gdf_counties.plot(
    edgecolor="black",
    column="housing_median_age",
    cmap="coolwarm",
    ax=ax,
)

norm_agg_housing_median_age = plt.Normalize(
    vmin=gdf_counties["housing_median_age"].min(),
    vmax=gdf_counties["housing_median_age"].max()
)

sm_agg_housing_median_age = plt.cm.ScalarMappable(
    norm=norm_agg_housing_median_age,
    cmap="coolwarm",
)

fig.colorbar(
    sm_agg_median_house_value, 
    ax=ax, 
    label="Housing Median Age"
    )

for x, y, name in zip(
    gdf_counties["centroid"].x, 
    gdf_counties["centroid"].y, 
    gdf_counties["name"]
    ):

    ax.text(x, y, name, fontsize=6, ha="center", va="center")

In [36]:
# Exportando a base de condados
gdf_counties.to_parquet(AGG_COUNTIES_DIR)

In [ ]:
# Gerando mapa interativo

import folium.plugins


map_center = [
    df["latitude"].mean(),
    df["longitude"].mean(), 
    ]

map_size_kw = {
    "width": 500, 
    "height": 500
    }

fig = folium.Figure(**map_size_kw)

map_obj = folium.Map(
    location=map_center,
    tiles="cartodb positron",
    zoom_start=5,
    control_scale=True,
    ).add_to(parent=fig)

# Obtendo conteúdo do geojson
with open(GEO_CALIFORNIA_ORIGINAL_DIR, 'r') as fp:

    geojson_content = json.load(fp)

folium.Choropleth(
    geo_data=geojson_content,
    data=gdf_counties,
    key_on="feature.properties.name",
    columns=["name", "median_house_value"],
    name="choropleth",
    legend_name="Valor Mediano das Casas",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.3,
).add_to(map_obj)


folium.GeoJson(
    data=gdf_counties[["name", "geometry", "median_income", "median_house_value"]],
    name="geojson",
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "median_income", "median_house_value"], 
        aliases=["Condado", "Renda Mediana Anual (milhares)", "Valor Mediano da Casa"])
).add_to(map_obj)

# Controle das camadas do mapa
folium.LayerControl().add_to(map_obj)

# Popup de latitude e longitude
#folium.LatLngPopup().add_to(map_obj)

# Plugin que recebe a posição do mouse no mapa (lat e long)
folium.plugins.MousePosition().add_to(map_obj)

map_obj